In [2]:
import pandas as pd
import csv
import time
import datetime as dt
import numpy as np

In [14]:
read_data = pd.read_csv('~/Research/Milind_Sohni_Research/data/cycles_shpr_sample.csv', delimiter = ',')


In [15]:
class Bus:
    """A class to define how many buses we need
    bus_no defines the number of the bus being used
    list_of_loops provides the index number of the loop
    this bus will travel on
    end_time provides the end time of the travel the bus 
    has done, i.e at what time bus will be free for another loop"""
    
      
    bus_no = 0
    end_time = dt.datetime(1900,1,1,0,0,0)
    
    def __init__(self, bus_no, end_time):
        """Initialize the bus_no and its end_time when you are declaring 
        a new bus is to be added to the system
        a new list, list_of_loops is added here because a list declared with bus_no
        will result in that list being shared with all the class objects, hence we 
        seperaly declare list_of_loops in constructor of the class, which localises it"""
        self.bus_no = bus_no
        self.end_time = end_time
        self.list_of_loops = []
    
    def add_a_loop_to_list_of_loops_of_a_bus(self, loop_no):
        """ To add a new loop to the list of loops already travelled by that bus"""
        self.list_of_loops.append(loop_no)
        

In [16]:
#for converting a string object to time object so that it can be used for future refernces
read_data['arrival_time']= pd.to_datetime(read_data['arrival_time'], format= '%H:%M:%S' )
read_data['departure_time']= (pd.to_datetime(read_data['departure_time'], format= '%H:%M:%S' ))


In [17]:
"""Declaring a list_of_buses with a list of Bus class objects
All the operations will be performed on this list of buses"""
list_of_buses = []

#Initialize the list_of_buses with the first route information so that comparisions can be done
list_of_buses.append(Bus(0,read_data.iloc[0,5]))
#adding loop 0 to the list of loops this bus is running on
list_of_buses[-1].add_a_loop_to_list_of_loops_of_a_bus(0)


In [18]:
#Printing out all the buses currently on the route
for bus in list_of_buses:
    print(bus.bus_no,bus.end_time)
    for bus_no in bus.list_of_loops:
        print('Loop number=', bus_no)

0 1900-01-01 05:15:00
Bus number= 0


In [19]:
"""
Checkpoint is declared so that we can traverse through 
all the buses to ensure that there is no bus which can 
acomodate the current loop bus_no counts the number of buses
we currently have.

First a single loop is selected(for i in read_data.index), 
then this loop is compared with all the current buses in the system
(for bus in list_of_buses).

If there is a bus which completes its loop before the current bus 
(if(read_data.iloc[i,4] >= bus.end_time and i != 0)) and is not 
the first bus, the bus is sent out on this route making the new 
loop's end time as this bus's end time and this loop is 
added to the list of loops of this bus. Checkpoint is made 0 to 
inform that this loop does not need a new bus.

After the (for bus in list_of_buses), we check if the checkpoint is 1
or not. If it is 1, it means that we need to add a new bus to this loop
as no other bus had compatible time for this loop.
"""

checkpoint = 0
#to keep track of number of buses needed
bus_no =0
for i in read_data.index:
    
    for bus in list_of_buses:
        #print(read_data.iloc[i,4] >= bus.end_time)
        #if the departure time of this loop is greater than the end time of the bus
        if(read_data.iloc[i,4] >= bus.end_time and i != 0):
            #increase bus end time to loop's end time
            bus.end_time = read_data.iloc[i,5]
            bus.add_a_loop_to_list_of_loops_of_a_bus(i)
            print("Checking when condition is true", bus.list_of_loops)
            
            #to inform that this loop does not need a new bus
            checkpoint = 0
            print('\nBus end time=', bus.end_time)
            print("\n")
            #no other buses are checked
            break
        #to check if this is not the first route, which has already been allocated a bus
        elif (i == 0):
            print('0')
            continue
        else:
            #informing that this loop might need a new bus
            print("else arrival time =", read_data.iloc[i,4])
            checkpoint = 1
        
        print(read_data.iloc[i,4] >= bus.end_time)
        
    #no bus was found compatible to this loop, a new bus needs to be added    
    if (checkpoint == 1):
        bus_no = bus_no + 1
        list_of_buses.append(Bus(bus_no,read_data.iloc[i,5]))
        print("appending new bus with end time", list_of_buses[-1].end_time)
        list_of_buses[-1].add_a_loop_to_list_of_loops_of_a_bus(i)
        print("Checking when a new bus is added", list_of_buses[-1].list_of_loops)
        print("\n")


0
else arrival time = 1900-01-01 04:30:00
False
appending new bus with end time 1900-01-01 05:00:00
Checking when a new bus is added [1]


Checking when condition is true [0, 2]

Bus end time= 1900-01-01 10:00:00


else arrival time = 1900-01-01 05:30:00
False
Checking when condition is true [1, 3]

Bus end time= 1900-01-01 07:05:00


else arrival time = 1900-01-01 05:30:00
False
else arrival time = 1900-01-01 05:30:00
False
appending new bus with end time 1900-01-01 11:45:00
Checking when a new bus is added [4]




In [20]:
for bus in list_of_buses:
    print(bus.bus_no,bus.end_time)
    for bus_no in bus.list_of_loops:
        print('Bus number=', bus_no)

0 1900-01-01 10:00:00
Bus number= 0
Bus number= 2
1 1900-01-01 07:05:00
Bus number= 1
Bus number= 3
2 1900-01-01 11:45:00
Bus number= 4
